In [1]:
import pandas as pd
import json
from pathlib import Path
import requests
from time import sleep
import sqlite3 as sql
import numpy as np

In [2]:
# API Configurations
HOST = "https://app.tabia.health/apiv1"
API_KEY = "ef823bc0-2381-4f2f-86fa-95c7caeba36d"

headers = {
    'accept': 'application/json',
    'api-key': API_KEY
}

In [3]:
patient_url = "/patient?size=9999999"
event_url = "/event?size=999999"
program_url = "/program?size=99999"


url = f"{HOST}{patient_url}"
print(f"json response:    {requests.get(url, headers=headers).json()}")
df_patients = pd.json_normalize(requests.get(url, headers=headers).json()["content"])
sleep(0.01)

url = f"{HOST}{event_url}"
print(f"json response:    {requests.get(url, headers=headers).json()}")
df_events = pd.json_normalize(requests.get(url, headers=headers).json()["content"])
sleep(0.01)

url = f"{HOST}{program_url}"
print(f"json response:    {requests.get(url, headers=headers).json()}")
df_programs = pd.json_normalize(requests.get(url, headers=headers).json()["content"])
sleep(0.01)

json response:    {'content': [{'id': 289091, 'fullName': 'Silva Costa', 'birthDate': '2025-01-20', 'ownContactData': False, 'email': '', 'since': '2025-01-20T16:52:46.294754Z', 'tags': [], 'healthInsurances': [], 'contacts': [], 'individualCarePlans': [{'id': 82646, 'patient': {'id': 289091, 'name': 'Silva Costa'}, 'pathway': {'id': 666, 'name': 'Módulo 03: Cirurgia'}, 'pathwayStartDate': '2025-01-20', 'pathwayStartReason': {}, 'pathwayEndReason': {}, 'bodyPart': {}, 'creationDate': '2025-01-20T16:58:03.8428Z'}, {'id': 82998, 'patient': {'id': 289091, 'name': 'Silva Costa'}, 'pathway': {'id': 1145, 'name': 'Módulo 03: Radioterapia'}, 'pathwayStartDate': '2025-01-22', 'pathwayStartReason': {}, 'pathwayEndReason': {}, 'bodyPart': {}, 'creationDate': '2025-01-22T14:19:32.289902Z'}], 'deviation': {'average': 18.2, 'total': 91.0, 'events': 5}}, {'id': 305562, 'fullName': 'teste próstata baixo risco 07-02', 'biologicalSex': 'male', 'birthDate': '1968-07-12', 'ownContactData': False, 'email'

In [4]:
# filename_events = "./event.json"
# filename_patients = "./patient.json"
filename_planos_de_cuidado = "./planos-de-cuidado.csv"
# filename_programs = "program.json"

# with open(filename_events, 'r') as file:
#     data = json.load(file)
#     df_events = pd.json_normalize(data["content"])

# with open(filename_patients, 'r') as file:
#     data = json.load(file)
#     df_patients = pd.json_normalize(data["content"])

with open(filename_planos_de_cuidado, 'r') as file:
    df_planos_de_cuidado = pd.read_csv(filename_planos_de_cuidado)

# with open(filename_programs, 'r') as file:
#     data = json.load(file)
#     df_programs = pd.json_normalize(data["content"])


# Consulta ENO

In [5]:
# Objetivo: planos-de-cuidado.csv ter uma só linha de paciente por programa de fato (API)
# Passo 1: Guardar o ID de todos os programas (https://app.tabia.health/apiv1/program)

program_names = list(df_programs["name"])
program_ids = list(df_programs["id"])

program_id_name_dict = {id:name for id, name in zip(program_ids, program_names)}

# Passo 2: Iterar sobre a URL (https://app.tabia.health/apiv1/program/{ID}/pathways) e ir montando um dicionário programa :[pathways]

program_pathways = {}

for program in program_id_name_dict.keys():
    url_by_id = f"{HOST}/program/{program}/pathways"
    print(f"'{program_id_name_dict[program]}' PATHWAYS:    {requests.get(url_by_id, headers=headers).json()}")
    response_dataframe = pd.json_normalize(requests.get(url_by_id, headers=headers).json())
    program_pathways[program] = list(response_dataframe["id"])
    sleep(0.01)

# Passo 3: Iterar pelo planos-de-cuidado.csv, adicionando a coluna "program_id" de acordo com a respectiva pathway

df_planos_de_cuidado["program"] = pd.Series()

index = 0
for row in df_planos_de_cuidado.itertuples(index=False):
    pathway_id = row[4]
    program = 0

    for program_id in program_id_name_dict.keys():
        # print(pathway_id, program_pathways[program_id], pathway_id in program_pathways[program_id])
        if pathway_id in program_pathways[program_id]:
            program = program_id
            df_planos_de_cuidado.loc[index, "program"] = program
    index += 1

# Passo 4: Remover rows que tem duplicates ao olhar para (patient_id && program_id)
df_planos_de_cuidado_unique = df_planos_de_cuidado.drop_duplicates(subset=["program", "patient_id"])

# Passo 5: Remover rows com os programas que os clientes não querem incluir na análise
values_to_remove = [119, 83, 229]
df_planos_de_cuidado_unique_filtered = df_planos_de_cuidado_unique[~df_planos_de_cuidado_unique["program"].isin(values_to_remove)]
df_planos_de_cuidado_unique_filtered

'01. Estadiamento' PATHWAYS:    [{'id': 693, 'name': '10 MAMA Módulo 02: Estadiamento', 'severityLevel': 1, 'totalPatientsCount': 14, 'activePatientsCount': 11, 'startDateExplanation': 'Consulta com oncologista clínico', 'endDateExplanation': 'Data em que o paciente finalizou o plano de cuidado', 'objective': 'SPEED', 'archived': False, 'icds': []}, {'id': 723, 'name': '20 PRÓSTATA Módulo 02: Estadiamento', 'severityLevel': 1, 'totalPatientsCount': 7, 'activePatientsCount': 5, 'startDateExplanation': 'Consulta com oncologista clínico', 'endDateExplanation': 'Data em que o paciente finalizou o plano de cuidado', 'objective': 'SPEED', 'archived': False, 'icds': []}, {'id': 797, 'name': '30 CÓLON E RETO Módulo 02: Estadiamento', 'severityLevel': 1, 'totalPatientsCount': 2, 'activePatientsCount': 2, 'startDateExplanation': 'Consulta com oncologista clínico', 'endDateExplanation': 'Data em que o paciente finalizou o plano de cuidado', 'objective': 'SPEED', 'archived': False, 'icds': []}, {'

,id,patient_id,patient_name,professionals,pathway_id,pathway_name,start_date,end_date,skip_until,body_part_id,body_part_name,body_part_archived,program
1,26848,232811,Juliana Pereira,NaN,694,Módulo 02: Plano de Tratamento,2024-11-13,2024-11-13,NaN,NaN,NaN,NaN,62
5,26899,232817,Alana Silva,NaN,694,Módulo 02: Plano de Tratamento,2024-11-18,NaN,NaN,NaN,NaN,NaN,62
9,32350,232811,Juliana Pereira,NaN,953,Módulo 04: Terapia Sistêmica,2024-11-27,2024-11-27,NaN,NaN,NaN,NaN,181
11,51356,232817,Alana Silva,NaN,485,Módulo 03: Radioterapia,2024-11-28,NaN,NaN,NaN,NaN,NaN,63
17,51850,232818,Patrícia Starling,NaN,1063,Módulo 02: Plano de Tratamento,2024-12-03,NaN,NaN,NaN,NaN,NaN,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,106530,312596,Teste colon e reto 26/02,NaN,1335,Módulo 05: Seguimento com amputação abdominope...,2025-02-26,NaN,NaN,NaN,NaN,NaN,160
207,106547,315031,Trastuzumabe,NaN,1342,Módulo 06: Seguimento,2025-02-27,NaN,NaN,NaN,NaN,NaN,62
209,106616,233039,Leonardo Domingos Teste,NaN,695,Módulo 02: Plano de Tratamento,2025-02-27,NaN,NaN,NaN,NaN,NaN,63
211,106648,233039,Leonardo Domingos Teste,NaN,772,Módulo 04: Radioterapia + Hormonioterapia,2025-02-27,NaN,NaN,NaN,NaN,NaN,133


In [6]:
def start_end_daterange(min_year, min_month, max_year, max_month):
    year = int(min_year)
    month = int(min_month)

    # end_year = int(max_year)
    # end_month = int(max_month) + 1
    # if end_month > 12:
    #     end_month = 1
    #     end_year =+ 1
    # print(end_year, end_month)
    result_readable = []
    result_parseable = []
    while True:
        result_readable.append(str(month) + "/" + str(year))
        result_parseable.append(str(year)+str(month))
        if year == int(max_year) and month == int(max_month):
            break
        month += 1
        if month > 12:
            year += 1
            month = 1
    return result_readable, result_parseable

In [7]:
def active_patients(df_target_planos_de_cuidado):
    # Só eventos alvo que me interessam / data_inicio / data_fim
    df_target_planos_de_cuidado = df_target_planos_de_cuidado.copy() # Pra remover um warning
    df_target_planos_de_cuidado["start-YYYYMM"] = (pd.to_datetime(df_target_planos_de_cuidado["start_date"]).dt.year.astype(str) + pd.to_datetime(df_target_planos_de_cuidado["start_date"]).dt.month.astype(str))
    df_target_planos_de_cuidado["end-YYYYMM"] = (pd.to_datetime(df_target_planos_de_cuidado["end_date"]).dt.year.fillna(-1)).astype(int).astype(str) + pd.to_datetime(df_target_planos_de_cuidado["end_date"]).dt.month.fillna(1).astype(int).astype(str)


    start_max_year, start_max_month = max(pd.to_datetime(df_target_planos_de_cuidado["start_date"])).year, max(pd.to_datetime(df_target_planos_de_cuidado["start_date"])).month
    start_min_year, start_min_month = min(pd.to_datetime(df_target_planos_de_cuidado["start_date"])).year, min(pd.to_datetime(df_target_planos_de_cuidado["start_date"])).month

    end_max_year, end_max_month = max(pd.to_datetime(df_target_planos_de_cuidado["end_date"])).year, max(pd.to_datetime(df_target_planos_de_cuidado["end_date"])).month
    end_min_year, end_min_month = min(pd.to_datetime(df_target_planos_de_cuidado["end_date"])).year, min(pd.to_datetime(df_target_planos_de_cuidado["end_date"])).month

    # print(f"start: {start_min_year}/{start_min_month}, end: {end_max_year}/{end_max_month}")

    # Corrigindo os nulls das end_dates
    if end_min_year not in range (1, 3000):
        end_min_year = start_min_year

    if end_min_month not in range (1, 13):
        end_min_month = start_min_month

    # Escolhendo o maior range possível, para abragir todas as datas de start e end
    # if max(df_target_planos_de_cuidado["end-YYYYMM"]) > max(df_target_planos_de_cuidado["start-YYYYMM"]):
    if max(pd.to_datetime(df_target_planos_de_cuidado["end_date"])) > max(pd.to_datetime(df_target_planos_de_cuidado["start_date"])):
        datas1 = {data:0 for data in start_end_daterange(start_min_year, start_min_month, end_max_year, end_max_month)[1]}
        print(start_min_year, start_min_month, end_max_year, end_max_month)
        datas2 = {data:0 for data in start_end_daterange(start_min_year, start_min_month, end_max_year, end_max_month)[1]}

    else:
        datas1 = {data:0 for data in start_end_daterange(start_min_year, start_min_month, start_max_year, start_max_month)[1]}
        print(start_min_year, start_min_month, start_max_year, start_max_month)
        datas2 = {data:0 for data in start_end_daterange(start_min_year, start_min_month, start_max_year, start_max_month)[1]}


    df_target_planos_de_cuidado = df_target_planos_de_cuidado.sort_values(by=["start-YYYYMM"])

    acumulativo = 0
    for data in datas1.keys():
        datas1[data] = sum(df_target_planos_de_cuidado["start-YYYYMM"] == data)


    df_target_planos_de_cuidado = df_target_planos_de_cuidado.sort_values(by=["end-YYYYMM"])

    for data in datas1.keys():
        datas2[data] = sum(df_target_planos_de_cuidado["end-YYYYMM"] == data)




    datas3 = pd.Series(datas1).cumsum()

    datas4 = pd.Series(datas2).cumsum()
    


    resultado = pd.DataFrame(
            {
                "starts":datas1,
                "ends":datas2,
                "start_cumulative":datas3,
                "end_cumulative":datas4
            }
        ).reset_index()
    
    resultado = resultado.rename(columns={"index" : "date"})

    resultado.loc[:, 'date'] = resultado['date'].astype(str).str[:4] + '-' + resultado['date'].astype(str).str[4:].str.zfill(2)

    resultado["active_patients"] = resultado["start_cumulative"].copy()

    for i in range(1, len(resultado)):
        resultado.loc[i, "active_patients"] = resultado.loc[i, "start_cumulative"] - resultado.loc[i - 1, "end_cumulative"]
    return resultado


In [8]:
dataframe_pacientes_ativos = active_patients(df_planos_de_cuidado_unique)
dataframe_pacientes_ativos_filtered = active_patients(df_planos_de_cuidado_unique_filtered)

2024 10 2025 2
2024 11 2025 2


In [22]:
consultas_enfermeira = df_events[df_events.name == "Consulta com Enfermeira Navegadora"]
consultas_enfermeira_pt = consultas_enfermeira[consultas_enfermeira["pathway.name"] == "Módulo 02: Plano de Tratamento"]
consultas_enfermeira_pt = consultas_enfermeira_pt.rename(columns={"originalTargetDate" : "start_date", "effectiveDate" : "end_date"})
dataframe_eventos_consulta = active_patients(consultas_enfermeira_pt)

indicador_ENO = dataframe_pacientes_ativos_filtered.merge(dataframe_eventos_consulta[["date", "end_cumulative"]], on='date', how='left')
indicador_ENO.rename(columns={"end_cumulative_y" : "consultas"}, inplace=True)
indicador_ENO2 = indicador_ENO[["date", "active_patients", "consultas"]]
indicador_ENO2.loc[:, "consultas"] = indicador_ENO2["consultas"].fillna(0)
INDICADOR_ENO_FINAL = indicador_ENO2
INDICADOR_ENO_FINAL

2024 11 2025 2


,date,active_patients,consultas
0,2024-11,4,1
1,2024-12,13,1
2,2025-01,26,1
3,2025-02,61,4


# Contagem de Idas à Emergência (não está cumulativo)

In [10]:
eventos_alvo = ["Pronto Socorro", "Intercorrência: Pronto Socorro"]
eventos_pronto_socorro = df_events[df_events.name.isin(eventos_alvo)]
eventos_pronto_socorro = eventos_pronto_socorro.rename(columns={"originalTargetDate" : "start_date", "effectiveDate" : "end_date"})
eventos_pronto_socorro

,id,name,categories,recommendations,targetDate,targetDateTemporary,start_date,end_date,scheduledAt,closed,...,formData.TratProp2,formData.ExAd,formData.Itensnaoprev:,formData.consultas,formData.checklistCirurgia,formData.itensBundle,formData.consultaBundle,formData.repetirBiopsia,formData.termoConsentimento,formData.fluxoValidacao
8,262269,Pronto Socorro,[],[],2025-02-27,False,2025-02-27,2025-02-27,2025-02-27,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,262470,Intercorrência: Pronto Socorro,[],[],2025-02-27,False,2025-02-27,2025-02-27,2025-02-27,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,79399,Pronto Socorro,[],[],2024-10-09,False,2024-10-09,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
dataframe_eventos = active_patients(eventos_pronto_socorro)

2024 10 2025 2


In [21]:
# Objetivo: Combinar dataframe de eventos mensal com o output do active_patients()
# Passo 1: Converter os 2 para dicionário de listas usando .to_dict(orient='list)
# Passo 2: Criar um terceiro dict, com as chaves que me interessam e suas respectivas listas. Montar o dataframe final
# Mais fácil do que eu pensava




# LEMBRAR QUE O EVENTO DE PRONTO SOCORRO SÓ ACONTECE NO 'ENDS', ISSO AQUI É SÓ PRA TER ALGUM DADO!! TESTAR

indicador_EMERGENCIA = dataframe_pacientes_ativos.merge(dataframe_eventos[["date", "starts"]], on='date', how='left')
indicador_EMERGENCIA.rename(columns={"starts_y" : "eventos_emergência"}, inplace=True)
indicador_EMERGENCIA2 = indicador_EMERGENCIA[["date", "active_patients", "eventos_emergência"]]
indicador_EMERGENCIA2.loc[:, "eventos_emergência"] = indicador_EMERGENCIA2["eventos_emergência"].fillna(0)
INDICADOR_EMERGENCIA_FINAL = indicador_EMERGENCIA2
INDICADOR_EMERGENCIA_FINAL

,date,active_patients,eventos_emergência
0,2024-10,1,1
1,2024-11,8,0
2,2024-12,29,0
3,2025-01,58,0
4,2025-02,112,2


# Interrupção TTO

In [13]:
# Objetivo: Montar o Interrupção TTO (API)
# Passo 1: Pegar o ID de pacientes que terminaram linha de tratamento através do planos-de-cuidado.csv
unique_filtered_finished_pathways = df_planos_de_cuidado_unique_filtered[~df_planos_de_cuidado_unique_filtered["end_date"].isna()]
unique_filtered_finished_pathways_patient_ids = list(unique_filtered_finished_pathways["patient_id"])

In [14]:
unique_filtered_finished_pathways_patient_ids

[232811, 232811, 263232, 309194, 309194, 309245, 301420, 233039, 233039]

In [15]:
# Passo 2: Iterar por /apiv1/individual-care-plan?patients={id}, montando a tabela razão_interrupcao/data_inicio/data_termino
ended_pathway_reasons = pd.DataFrame(
                        {
                            "razao_interrupcao": [],
                            "data_inicio" : [],
                            "data_termino" : []
                        }
                    )

for patient_id in unique_filtered_finished_pathways_patient_ids[::-1]:
    url_by_id = f"{HOST}/individual-care-plan?patients={patient_id}"
    print(f"http response:    {requests.get(url_by_id, headers=headers).json()}")
    response_dataframe = pd.json_normalize(requests.get(url_by_id, headers=headers).json()["content"])
    
    # Se o paciente não tem um endReason, o dataframe dele fica diferente. Consertando isso aqui
    if "pathwayEndReason.name" not in response_dataframe.columns:
        response_dataframe["pathwayEndReason.name" ] = pd.Series()
    
    response_dataframe = response_dataframe.rename(columns={
        "pathwayEndReason.name" : "razao_interrupcao",
        "pathwayStartDate":"start_date",
        "pathwayEndDate":"end_date"
        })
    useful_response_info = response_dataframe[["razao_interrupcao", "start_date", "end_date"]]
    ended_pathway_reasons = pd.concat([ended_pathway_reasons, useful_response_info], ignore_index=True)
    sleep(0.01)

http response:    {'content': [{'id': 53124, 'patient': {'id': 233039, 'name': 'Leonardo Domingos Teste'}, 'professionalTypes': [], 'pathway': {'id': 982, 'name': '40 PULMÃO Módulo 01: Diagnóstico', 'program': {'id': 83, 'name': '00. Diagnóstico'}, 'createdAt': '2024-11-13', 'startDateExplanation': 'Base para cálculo dos eventos no plano de cuidado', 'endDateExplanation': 'Data em que o paciente finalizou o plano de cuidado', 'objective': 'SPEED', 'archived': False, 'professionalActivities': [], 'icds': []}, 'pathwayStartDate': '2024-12-05', 'pathwayEndDate': '2025-02-25', 'pathwayEndReason': {'id': 134, 'name': 'Desistência (escolha pessoal)'}, 'creationDate': '2024-12-05T14:03:48.114734Z'}, {'id': 53125, 'patient': {'id': 233039, 'name': 'Leonardo Domingos Teste'}, 'professionalTypes': [], 'pathway': {'id': 983, 'name': '40 PULMÃO Módulo 02: Estadiamento', 'program': {'id': 119, 'name': '01. Estadiamento'}, 'createdAt': '2024-11-13', 'severityLevel': 1, 'startDateExplanation': 'Consu

In [16]:
# Passo 4: Iterar por essa lista usando active_patients() com um filtro por razão (aqui só o end vai importar)
ended_pathway_reasons["razao_interrupcao"].fillna("Não especificada", inplace=True)
razoes_interrupcao = ended_pathway_reasons["razao_interrupcao"].unique()

analises_interrupcao = []
for razao in razoes_interrupcao:
    analise = active_patients(ended_pathway_reasons[ended_pathway_reasons["razao_interrupcao"] == razao])
    analise["Reason"] = razao
    analises_interrupcao.append(analise)


2024 12 2025 2
2024 12 2025 2
2024 10 2025 2


In [20]:
# Todos estão em uma lista. Ver depois na mensagem como o Édison prefere os dados organizados
# active_patients tem 2 colunas?? e nenhumas dela é a correta
indicador_tto = pd.concat(analises_interrupcao, ignore_index=True)
indicador_tto.drop(columns=["starts", "start_cumulative", "ends", "active_patients"], inplace=True)

# pegando o active_patients do ENO que já funcionava
indicador_tto2 = indicador_tto.merge(INDICADOR_ENO_FINAL[["date", "active_patients"]], on='date', how='left')

indicador_ENO = dataframe_pacientes_ativos_filtered.merge(dataframe_eventos_consulta[["date", "end_cumulative"]], on='date', how='left')

# indicador_tto2["active_patients"].fillna(0, inplace=True)    SE NAO TIVER O DADO CHAVE QUE VOU COLOCAR (CUMULATIVE ENDS) TIRAR
INDICADOR_INTERRUPCAO_FINAL = indicador_tto2[indicador_tto2["end_cumulative"] != 0]
INDICADOR_INTERRUPCAO_FINAL

,date,end_cumulative,Reason,active_patients
2,2025-02,7,Desistência (escolha pessoal),61.0
5,2025-02,4,Óbito,61.0
7,2024-11,6,Não especificada,4.0
8,2024-12,6,Não especificada,13.0
9,2025-01,7,Não especificada,26.0
10,2025-02,14,Não especificada,61.0


In [18]:
# DONE - Objetivo: planos-de-cuidado.csv ter uma só linha de paciente por programa de fato (API)

# Passo 1: Guardar o ID de todos os programas (https://app.tabia.health/apiv1/program)
# Passo 2: Iterar sobre a URL (https://app.tabia.health/apiv1/program/{ID}/pathways) e ir montando um dicionário programa :[pathways]
# Passo 3: Iterar pelo planos-de-cuidado.csv, adicionando a coluna "program_id" de acordo com a respectiva pathway
# Passo 4: Remover rows que tem duplicates ao olhar para (patient_id && program_id)
# Passo 5: Remover rows com os programas que os clientes não querem incluir na análise


# DONE - Objetivo: Combinar dataframe de eventos mensal com o output do active_patients()

# Passo 1: Converter os 2 para dicionário de listas usando .to_dict(orient='list)
# Passo 2: Criar um terceiro dict, com as chaves que me interessam e suas respectivas listas. Montar o dataframe final


# Objetivo: Montar o Interrupção TTO (API)

# Passo 1: Pegar o ID de pacientes que terminaram linha de tratamento através do planos-de-cuidado.csv
# Passo 2: Iterar por /apiv1/individual-care-plan?patients={id}, montando a tabela razão_interrupcao/data_inicio/data_termino
# Passo 3: Montar uma lista com as razões de interrupção
# Passo 4: Iterar por essa lista usando active_patients() com um filtro por razão (aqui só o end vai importar)
# Passo 5: Desmembrar os diferentes dataframes usando .to_dict(orient='list) e juntar tudo em 1 só
# Passo 6: (Opcional) Usar SQL para fazer um GROUP BY pelos nomes de profissionais


# Objetivo: Montar o Produção por ENO

# Passo 1: Pegar todos os ENOs pela URL (https://app.tabia.health/apiv1/professional) filtrando pelas professional_activities, só salvando ENO
# Passo 2: Separar rows do planos-de-cuidado.csv pela quantidade de profissionais que ela tem, pra ficar só 1 por linha
# Passo 3: Remover rows que tem duplicates ao olhar para (patient_id && program_id && profissional) || profissional != ENO
# Passo 4: Montar uma lista com os profissinais que ainda estão presentes nesse csv
# Passo 5: Iterar nesse csv para cada um desses profissionais usando active_patients() com um filtro por profissional
# Passo 6: Desmembrar todos os diferentes dataframes usando .to_dict(orient='list') e juntar tudo de em 1 só
# Passo 7: (Opcional) Usar SQL para fazer um GROUP BY pelos nomes de profissionais

# Produção por enfermeiro navegador

In [19]:
# Precisamos da lista de enfermeiros navegadores, ou eles vão ter que ter um identificador no nome (assim como o doutor)
# O ideal é na verdade o sistema da tabia mostrar o role dos professionals

df_planos_de_cuidado_unique[df_planos_de_cuidado_unique["professionals"].notna()]

,id,patient_id,patient_name,professionals,pathway_id,pathway_name,start_date,end_date,skip_until,body_part_id,body_part_name,body_part_archived,program
6,26956,232814,Pedro Carlos,Dra. Milena Silva,982,40 PULMÃO Módulo 01: Diagnóstico,2024-11-19,NaN,NaN,NaN,NaN,NaN,83
90,96525,302936,Maria Rita 05/02,Leonardo Domingos,1241,50 RIM Módulo 01: Diagnóstico,2025-02-05,NaN,NaN,NaN,NaN,NaN,83
191,106082,233039,Leonardo Domingos Teste,"Dr. Rafael Souza, Leonardo Domingos",657,NAVG - Terapia Sistêmica,2025-02-24,2025-02-25,NaN,NaN,NaN,NaN,112
213,106654,260165,CAM,Leonardo Domingos,1285,Módulo 02: Plano de Tratamento,2025-02-27,NaN,NaN,NaN,NaN,NaN,254
214,106655,315154,Alcides Joaquim,"Livia Teixeira, Raquel Alessio",723,20 PRÓSTATA Módulo 02: Estadiamento,2025-02-27,NaN,NaN,NaN,NaN,NaN,119
